In [2]:
import pandas as pd
import numpy as np
import csv
import time
import sqlite3
import random
import math
import time
from tqdm import tqdm

In [3]:
# 用户听过艺术次数文件
user_rate_file = "/Users/user/Downloads/hetrec2011-lastfm-2k (1)/user_artists.dat"
# 用户打标签信息
user_tag_file = "/Users/user/Downloads/hetrec2011-lastfm-2k (1)/user_taggedartists.dat"

In [4]:
# 获取所有的艺术家ID
artistAll = list(pd.read_csv("/Users/user/Downloads/hetrec2011-lastfm-2k (1)/artists.dat",delimiter='\t')['id'].values)

In [6]:
len(artistAll)

17632

In [7]:
# 获取用户对艺术家的评分信息
def getUserRate():
    userRateDict = dict()
    fr = open(user_rate_file, "r", encoding="utf-8")
    for line in fr.readlines():
        if not line.startswith("userID"):
            userID, artistID, weight = line.split("\t")
            userRateDict.setdefault(int(userID), {})
            # 对听歌次数进行数据归一化，避免计算结果过大
            userRateDict[int(userID)][int(artistID)] = float(weight) / 10000
    return userRateDict

In [8]:
userRateDict = getUserRate()

In [9]:
userRateDict

{2: {51: 1.3883,
  52: 1.169,
  53: 1.1351,
  54: 1.03,
  55: 0.8983,
  56: 0.6152,
  57: 0.5955,
  58: 0.4616,
  59: 0.4337,
  60: 0.4147,
  61: 0.3923,
  62: 0.3782,
  63: 0.3735,
  64: 0.3644,
  65: 0.3579,
  66: 0.3312,
  67: 0.3301,
  68: 0.2927,
  69: 0.272,
  70: 0.2686,
  71: 0.2654,
  72: 0.2619,
  73: 0.2584,
  74: 0.2547,
  75: 0.2397,
  76: 0.2382,
  77: 0.212,
  78: 0.2119,
  79: 0.199,
  80: 0.1972,
  81: 0.1948,
  82: 0.1868,
  83: 0.1792,
  84: 0.174,
  85: 0.1638,
  86: 0.1594,
  87: 0.1559,
  88: 0.1553,
  89: 0.1519,
  90: 0.1471,
  91: 0.1438,
  92: 0.1411,
  93: 0.1407,
  94: 0.1373,
  95: 0.1363,
  96: 0.1342,
  97: 0.1337,
  98: 0.1332,
  99: 0.133,
  100: 0.1315},
 3: {101: 1.3176,
  102: 0.0662,
  103: 0.0493,
  104: 0.0431,
  105: 0.0403,
  106: 0.0354,
  107: 0.0269,
  108: 0.0236,
  109: 0.0215,
  110: 0.0215,
  111: 0.0212,
  112: 0.0193,
  113: 0.0187,
  114: 0.0167,
  115: 0.0163,
  116: 0.0154,
  117: 0.0151,
  118: 0.0142,
  119: 0.0142,
  120: 0.0125,


In [12]:
# 获取艺术家对应的标签基因,这里的相关度全部为1
# 由于艺术家和tag过多，存储到一个矩阵中维度太大，这里优化存储结构
# 如果艺术家有对应的标签则记录，相关度为1，否则不为1
def getArtistsTags():
    artistsTagsDict = dict()
    for line in open(user_tag_file, "r", encoding="utf-8"):
        if not line.startswith("userID"):
            artistID, tagID = line.split("\t")[1:3]
            artistsTagsDict.setdefault(int(artistID), {})
            artistsTagsDict[int(artistID)][int(tagID)] = 1
    return artistsTagsDict

In [13]:
artistsTagsDict = getArtistsTags()

In [14]:
artistsTagsDict

{52: {13: 1,
  15: 1,
  18: 1,
  21: 1,
  41: 1,
  39: 1,
  130: 1,
  20: 1,
  24: 1,
  557: 1,
  727: 1,
  826: 1,
  1492: 1,
  5083: 1,
  6794: 1,
  6796: 1,
  6808: 1,
  102: 1,
  238: 1,
  8730: 1,
  4847: 1,
  127: 1,
  2336: 1,
  10534: 1,
  11083: 1,
  11613: 1,
  184: 1},
 63: {13: 1,
  14: 1,
  23: 1,
  40: 1,
  18: 1,
  801: 1,
  825: 1,
  835: 1,
  836: 1,
  845: 1,
  1492: 1,
  1495: 1,
  2101: 1,
  79: 1,
  134: 1,
  370: 1,
  727: 1,
  2362: 1,
  191: 1,
  6242: 1,
  229: 1,
  6444: 1,
  6445: 1,
  1032: 1,
  7113: 1,
  187: 1,
  352: 1,
  7243: 1,
  238: 1,
  309: 1,
  3467: 1,
  12530: 1,
  8024: 1},
 73: {13: 1,
  14: 1,
  15: 1,
  18: 1,
  20: 1,
  21: 1,
  22: 1,
  26: 1,
  78: 1,
  79: 1,
  83: 1,
  85: 1,
  130: 1,
  38: 1,
  11499: 1,
  234: 1,
  299: 1,
  825: 1,
  1492: 1,
  12092: 1,
  12093: 1,
  12095: 1},
 94: {13: 1, 15: 1, 20: 1, 21: 1, 22: 1, 36: 1, 37: 1, 39: 1, 850: 1},
 96: {19: 1,
  24: 1,
  201: 1,
  1096: 1,
  1098: 1,
  192: 1,
  735: 1,
  1354: 1,

In [18]:
# 获取每个用户打标的标签和每个标签被所有用户打标的次数
def getUserTagNum():
    userTagDict = dict()
    tagUserDict = dict()
    for line in open(user_tag_file, "r", encoding="utf-8"):
        if not line.startswith("userID"):
            userID, artistID, tagID = line.strip().split("\t")[:3]
            # 统计每个标签被打标的次数
            if int(tagID) in tagUserDict.keys():
                tagUserDict[int(tagID)] += 1
            else:
                tagUserDict[int(tagID)] = 1
            # 统计每个用户对每个标签的打标次数
            userTagDict.setdefault(int(userID), {})
            if int(tagID) in userTagDict[int(userID)].keys():
                userTagDict[int(userID)][int(tagID)] += 1
            else:
                userTagDict[int(userID)][int(tagID)] = 1
    return userTagDict

In [19]:
userTagDict = getUserTagNum()

In [20]:
userTagDict

{2: {13: 5,
  15: 4,
  18: 3,
  21: 3,
  41: 1,
  14: 4,
  23: 1,
  40: 1,
  20: 3,
  22: 2,
  26: 1,
  36: 1,
  37: 1,
  39: 1,
  19: 1,
  24: 2,
  16: 2,
  17: 1,
  25: 1,
  42: 1,
  43: 1,
  32: 1,
  33: 1,
  34: 1,
  38: 1,
  35: 1},
 3: {14: 6,
  15: 4,
  33: 7,
  44: 5,
  45: 2,
  46: 4,
  47: 2,
  48: 1,
  49: 2,
  50: 1,
  51: 1,
  52: 2,
  53: 2,
  54: 1,
  55: 2,
  56: 1,
  57: 1,
  58: 1,
  63: 1,
  64: 1,
  65: 1,
  68: 1,
  71: 1,
  18: 1,
  69: 1,
  34: 2,
  66: 1,
  70: 1},
 4: {16: 1,
  25: 2,
  18: 8,
  13: 1,
  73: 5,
  14: 2,
  75: 2,
  74: 2,
  79: 2,
  24: 6,
  77: 1,
  93: 1,
  80: 1,
  78: 3,
  88: 1,
  72: 3,
  82: 1,
  85: 1,
  76: 4,
  84: 2,
  87: 1,
  89: 1,
  1: 1,
  91: 1,
  90: 2,
  86: 1,
  15: 1,
  92: 1,
  7: 1,
  6: 1,
  83: 1},
 5: {95: 14, 94: 9, 99: 4, 98: 4, 96: 7, 97: 5},
 6: {117: 1,
  118: 1,
  119: 1,
  120: 1,
  100: 1,
  101: 1,
  112: 1,
  113: 1,
  114: 1,
  121: 1,
  102: 1,
  103: 1,
  122: 1,
  123: 1},
 7: {127: 2, 128: 1},
 8: {24: 4,